# Analisis de incertidumbre

Notebook para analizar algoritmos cuando se tiene incertidumbre en el tiempo de partida.

En primer lugar definimos el algoritmo a estudiar (por ahora EDF o LLF)

In [1]:
Algorithm = "EDF";

In [2]:
#using Pkg;Pkg.instantiate()
#import Pkg; Pkg.add("Distributions")
using EVQueues, Plots, Distributions

Se definen los parámetros para realizar las simulaciones

In [3]:
#Desviación standard de la incertidumbre (se modela como una Gaussiana)
sigma=0.2

#Parámetros del resto de la simulación
lambda=60.0;
mu=1;
gamma=1;
C=40.0;
Tfinal=100.0;

#Se agrega snaps para hacer animaciones
snaps = collect(0.1:.5:100.0);

El próximo cuadro compara el desempeño promedio

El siguiente cuadro analiza resultados de vehiculos individualmente

In [4]:
df = generate_Poisson_stream(lambda,mu,gamma,Tfinal);

#Se corre el algoritmo "base", esto es, considerando los deadlines reales
if Algorithm == "EDF"
    alg = ev_edfc_trace(df,C,snapshots=snaps);
    compute_statistics!(alg);
end

if Algorithm == "LLF"
    alg = ev_llf_trace(df,C,snapshots=snaps);
    compute_statistics!(alg);
end

#Se perturban las salidas, y se simula considerandola como la salida reportada
perturbation = Normal(0,sigma)
reportedDeadlines = [d+rand(perturbation) for d in df[!,:salidas]]
df[!,:salidaReportada] = reportedDeadlines

lifo = ev_lifo_trace(df,C);
compute_statistics!(lifo);
lifo_evs = sort(lifo.EVs,by=ev->ev.arrivalTime)

if Algorithm == "EDF"
    alg_uncertain = ev_edfc_trace(df,C,snapshots=snaps);
    compute_statistics!(alg_uncertain);
end
    
if Algorithm == "LLF"
    alg_uncertain = ev_llf_trace(df,C,snapshots=snaps);
    compute_statistics!(alg_uncertain);
end

alg_evs = sort(alg.EVs,by=ev->ev.arrivalTime)
alg_uncertain_evs = sort(alg_uncertain.EVs,by=ev->ev.arrivalTime)

#para recortar el transitorio
n=Int(Tfinal*lambda/10);
alg_evs = alg_evs[n:end-n]
alg_uncertain_evs = alg_uncertain_evs[n:end-n]
lifo_evs = lifo_evs[n:end-n]

#Guardo resultados de vehiculos
S_alg = [ev.requestedEnergy for ev in alg_evs];
S_alg_uncertain = [ev.requestedEnergy for ev in alg_uncertain_evs];
S_lifo = [ev.requestedEnergy for ev in lifo_evs];

Sr_alg = [ev.departureWorkload for ev in alg_evs];
Sr_alg_uncertain = [ev.departureWorkload for ev in alg_uncertain_evs];
Sr_lifo = [ev.departureWorkload for ev in lifo_evs];

Sa_alg = S_alg - Sr_alg
Sa_alg_uncertain = S_alg_uncertain - Sr_alg_uncertain;
Sa_lifo = S_lifo - Sr_lifo;

Simulando... 100%|██████████████████████████████████████| Time: 0:00:01
Simulando... 100%|██████████████████████████████████████| Time: 0:00:00


4844-element Array{Float64,1}:
 1.0917325651301897
 0.47767857190565394
 0.16018805597668143
 0.40483521560553487
 0.3060404993859207
 0.16294009252165775
 0.97154369596989
 0.764211501389121
 0.0045847645312409365
 0.8324821670234042
 0.9242365726283406
 0.9699641024463106
 0.11492104300698443
 ⋮
 0.5695743544401535
 1.4614480892988282
 0.10260512893691463
 0.6950100561853106
 0.09846266855076795
 0.5704036419251336
 1.369814852502318
 0.6990142364560562
 0.34851420795395177
 1.3477106098455032
 0.33079624667541885
 0.04844508167212389

In [5]:
full_alg = [ev.departureWorkload==0 for ev in alg_evs];
full_alg_uncertain = [ev.departureWorkload==0 for ev in alg_uncertain_evs];
full_lifo = [ev.departureWorkload==0 for ev in lifo_evs];

a=full_lifo.*full_alg_uncertain
b=full_lifo.+full_alg_uncertain
both_full = [i>0 for i in a];
both_unfull = [i==0 for i in b];

p = plot(  xlabel="LIFO",
            ylabel=Algorithm * " Uncertain",
            title="Energy Received", legend=:bottomright)

scatter!(p,Sa_lifo[both_unfull.==true],Sa_alg_uncertain[both_unfull.==true],markershape=:circle,markersize=4,color=:blue,label="Not full")
scatter!(p,Sa_lifo[full_lifo.==true],Sa_alg_uncertain[full_lifo.==true],markershape=:circle,markersize=4,color=:red,label="Full LIFO")
scatter!(p,Sa_lifo[full_alg_uncertain.==true],Sa_alg_uncertain[full_alg_uncertain.==true],markershape=:circle,markersize=4,color=:yellow,label="Full Uncertain")
scatter!(p,Sa_lifo[both_full.==true],Sa_alg_uncertain[both_full.==true],markershape=:circle,markersize=4,color=:black,label="Both full")
savefig(p,"Energy Received LIFO vs EDF - Curtail - No Bias.pdf")

In [6]:
alg_departure = [ev.reportedDepartureTime for ev in alg_evs];
alg_uncertain_departure = [ev.reportedDepartureTime for ev in alg_uncertain_evs];
diff_departure=alg_departure-alg_uncertain_departure;

EV_overstayed = [i>0 for i in diff_departure]
EV_understayed = [i<0 for i in diff_departure]

p = plot(  xlabel=Algorithm,
            ylabel="LIFO",
            title="Energy Received", legend=:bottomright)

scatter!(p,Sa_lifo[both_full.==false],Sa_alg_uncertain[both_full.==false],markershape=:circle,markersize=4,marker_z=(diff_departure[both_full.==false]),colorbar=true,label=false)
savefig(p,"Energy requested vs received EDF - Curtail - No Bias - Markerz.pdf")

In [7]:
diff_Sa=Sa_alg_uncertain-Sa_lifo;
alg_departure = [ev.reportedDepartureTime for ev in alg_evs];
alg_uncertain_departure = [ev.reportedDepartureTime for ev in alg_uncertain_evs];
diff_departure=alg_departure-alg_uncertain_departure;

p = plot(  xlabel="Difference in real and reported departure",
            ylabel="Difference in energy received",
            title="Comparison between LIFO and" * Algorithm, legend=:bottomright)

scatter!(p,diff_departure[both_unfull.==true],diff_Sa[both_unfull.==true],markershape=:circle,markersize=4,color=:blue,label="Not full")
scatter!(p,diff_departure[full_lifo.==true],diff_Sa[full_lifo.==true],markershape=:circle,markersize=4,color=:red,label="Full EDF")
scatter!(p,diff_departure[full_alg_uncertain.==true],diff_Sa[full_alg_uncertain.==true],markershape=:circle,markersize=4,color=:yellow,label="Full Uncertain")
scatter!(p,diff_departure[both_full.==true],diff_Sa[both_full.==true],markershape=:circle,markersize=4,color=:black,label="Both full")
savefig(p,"Comparison between LIFO and EDF - Curtail - No Bias.pdf")

In [8]:
s1 = scatter(S_lifo,Sa_lifo,label="LIFO", ylabel="Energy received",markershape=:circle,markersize=4,color=:blue, legend=:bottomright)
s2 = scatter(S_alg_uncertain,Sa_alg_uncertain,label=Algorithm* " Uncertain",markershape=:circle,markersize=4,color=:red, legend=:bottomright)
plot(s1, s2, layout = (2, 1), xlabel="Energy requested", xlims = (0,5))
savefig("Energy requested vs received - Curtail - No Bias.pdf")